# Tracker Requests Positions from Vehicles

## 0.0 - Imports

In [139]:
import pandas as pd 
from requests import post 
import json, pickle

import sqlite3      # to create database
import sqlite_utils # to insert, update and delete information
from sqlalchemy import create_engine

from IPython.core.display import HTML
import matplotlib.pyplot as plt
import seaborn as sns

import streamlit as st

# pandas to transform data and convert to dataframe
# requests.post to invite data to serverless and the serverless response with object
# json to convert data to json or vice versa

### 0.1 - Helper Functions

In [140]:
    
def create_table(cursor, query):
    
    ''' Create table with SQLAlchemy from DataFrame object'''
    
    cursor.execute(query)


def jupyter_settings():
    
    
    %matplotlib inline
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25,12]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container{width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = 100
    pd.set_option('display.expand_frame_repr', False)

    sns.set()
jupyter_settings()


# Function to get user data and exported to pickle file.
# I deleted data inputed for security, it was only function to now how you done.
def user_logon():
    
    
    ''' Store the data to Authentication from integration Tracker
        url_logon: url to Authentication and get token
        url_veiculos: Url to load vehicles informations
        url_posicao: Url to load positions informations
        headers: Type of send headers this case is json
        handshake: Data to authentication from integration
                post (informations that you must send like handshake)
                    username: username account to access grid tracker
                    password: password account to access grid tracker
                    appid: Id delivered from Tracker
                response (informations that you receive after logon)
                    token: the alfanumeric code that it's must be send post
                    expiration: There aren't expiration '''
    
    url_logon = 'http://'
    url_veiculos = 'http://'
    url_posicao = 'http://'
    headers = {"content-type":"application/json"}
    handshake = {'username':'', 'password':'', 'appid':'', 'token':'', 'expriration':''}
    
    return url_logon, url_veiculos, url_posicao, headers, handshake

Populating the interactive namespace from numpy and matplotlib


### 0.2 - Requests Data Collect

Requests API from Tracker, with method Post, see the <a href="">documentation</a>

#### 0.2.1 - User Authentication

In [141]:
# Dumping object with data authentication to pickle format
# with open("user_logon.pkl", "wb") as file:
#    pickle.dump(user_logon(), file)


# Loading data from pickle file content user data to connection authentication
with open("user_logon.pkl", "rb") as f_load:
    url_logon, url_vehicles, url_position, headers, handshake = pickle.load(f_load)

Token

In [142]:
# send data and return object to store token
# without token can't access the vehicle
response_logon = post(url_logon,
                        data=json.dumps(handshake),
                        headers=headers) # response array object
token = {"token":response_logon.json()['object']
        ['token']} # store token form object

#### 0.2.2 - Vehicles

In [143]:
# Requests to get vehicle informations

response_vehicles = post(url_vehicles, headers=token)
obj_vehicle = response_vehicles.json() # transform obj array to json


'\nresponse_vehicles = post(url_vehicles, headers=token)\nobj_vehicle = response_vehicles.json() # transform obj array to json\n'

In [144]:
# Save json file in the local folder
# this only for development mode, after validation, it's better maker a 
# function to store object without saving
#with open('collected_2020061810.json', 'w') as json_file:
#    json.dump(obj_veiculo, json_file)

# Transforming Json to DataFrame
obj_vehicle_df = pd.DataFrame(obj_vehicle['object']).copy()

"obj_vehicle_df = pd.DataFrame(obj_vehicle['object']).copy()"

In [145]:
'''
# Drop List to delete columns that it's not necessary for while
drop_list = """
    codigorf          
    odometroGps        
    dataAquisicao 
    distanciaKmFrete
    kmManual
    horimetroManual
    horimetroAtual
    kmAtual
    statusVenda
    dataAtivado
    dataCancelado
    finalizado
    renavam
    dataInstalacao
    motoristas
    dispositivos
    grupos
    """.split()
df_vehicles = obj_vehicle_df.drop(drop_list, axis=1).copy()
'''

'\n# Drop List to delete columns that it\'s not necessary for while\ndrop_list = """\n    codigorf          \n    odometroGps        \n    dataAquisicao \n    distanciaKmFrete\n    kmManual\n    horimetroManual\n    horimetroAtual\n    kmAtual\n    statusVenda\n    dataAtivado\n    dataCancelado\n    finalizado\n    renavam\n    dataInstalacao\n    motoristas\n    dispositivos\n    grupos\n    """.split()\ndf_vehicles = obj_vehicle_df.drop(drop_list, axis=1).copy()\n'

#### 0.2.3 - Positions

In [146]:
# Requests to get vehicle positions
response_position = post(url_position, headers=token)
obj_position = response_position.json()

In [147]:
# Transforming Json to DataFrame
df_positions = pd.DataFrame(obj_position['object']).copy()

<h3>Resumo em Português</h3>
<p>realizado o request com método post para a api da tracker no post contém as informações de envio; o tipo de dados "json" os dados do usuário(user, senha, id) e retornado um objecto com as informações e enviadas e mais um objeto com as informações do token, sem o token não é possível fazer a requisição, com isso na requisição para veiculos é enviado o token para que deem acesso aos dados, então retornado um objeto array com os dados dos veiculos, porém mais da metade dos campos(colunas) estão vazias(nulas), com isso uma das informações mais importantes que é o posicionamento do veiculo estão nulas, invalidando o projeto, tem outra url de para requisição que é para pegar especificadamente o posicionamento, porém, retornado vazio também, será comunicado ao responsável técnico da tracker para correção ou permissão da mesma.(20/06/2020)</p>

#### 0.2.4 - Pickle loading to tests

In [148]:
#with open("df_vehicles.pkl", "rb") as file:
#    df_vehicles = pickle.load(file)
#
#with open("df_positions.pkl", "rb") as file:
#    df_positions = pickle.load(file)

## 1.0 - Data Description

### 1.1 - Rename Columns

### 1.2 - Data Dimention

In [149]:
df_vehicles.shape, df_positions.shape

((9955, 20), (2830, 37))

### 1.3 - Check NaN and Fillout

#### 1.3.1 - Vehicle DataFrame

In [150]:
# See all the columns and check the NaN
print(df_vehicles.isna().sum())

id                      0
dataCadastrado          0
fuso                 8241
deletado                0
status                  0
vin                     0
anoFabricacao           0
anoModelo               0
placa                   0
tipoMonitoramento    6106
marca                   0
modelo                  0
cor                     0
descricao               0
frota                9345
tipo                    0
assistencia             0
usuarioCriacao          0
proprietarioId          0
proprietario            0
dtype: int64


In [151]:
# list of columns that you will drop from DataFrame because is Null
drop_list_vehicles = [
    'fuso', 'tipoMonitoramento', 'frota'
]

In [152]:
df_vehicles.drop(drop_list_vehicles, axis=1).isna().sum()

id                0
dataCadastrado    0
deletado          0
status            0
vin               0
anoFabricacao     0
anoModelo         0
placa             0
marca             0
modelo            0
cor               0
descricao         0
tipo              0
assistencia       0
usuarioCriacao    0
proprietarioId    0
proprietario      0
dtype: int64

In [153]:
# Fillout the NaN to any values
df_vehicles['status'] = df_vehicles['status'].fillna('Ativado').copy()
df_vehicles['anoFabricacao'] = df_vehicles['anoFabricacao'].fillna(0).copy()
df_vehicles['anoModelo'] = df_vehicles['anoModelo'].fillna(0).copy()
df_vehicles['marca'] = df_vehicles['marca'].fillna(0).copy()
df_vehicles['cor'] = df_vehicles['cor'].fillna(0).copy()
df_vehicles['descricao'] = df_vehicles['descricao'].fillna('').copy()
df_vehicles['tipo'] = df_vehicles['tipo'].fillna('Caminhão').copy()

#### 1.3.2 - Positions DataFrame

In [154]:
# See all the columns and check the NaN
print(df_positions.isna().sum())

id                      0
codigorf             2830
odometroGps          2830
dataAquisicao        2830
distanciaKmFrete     2830
kmManual             2830
horimetroManual      2830
horimetroAtual       2830
kmAtual              2830
statusVenda          2830
dataAtivado          2830
dataCadastrado       2830
dataCancelado        2830
fuso                 2830
deletado             2830
status               2830
finalizado           2830
renavam              2830
vin                  2830
anoFabricacao        2830
anoModelo            2830
placa                   0
dataInstalacao       2830
tipoMonitoramento    2830
marca                2830
modelo               2830
cor                  2830
descricao            2830
frota                2830
tipo                 2830
assistencia             0
usuarioCriacao       2830
proprietarioId          0
proprietario         2830
motoristas              0
dispositivos            0
grupos                  0
dtype: int64


In [155]:
# list of columns that you will drop from DataFrame because is Null
drop_list_positions = [
    'codigorf', 'odometroGps', 'dataAquisicao', 'distanciaKmFrete',  
    'kmManual', 'horimetroManual', 'horimetroAtual', 'kmAtual',            
    'statusVenda', 'dataAtivado', 'dataCadastrado', 'dataCancelado', 
    'fuso', 'deletado', 'status', 'finalizado', 'renavam',           
    'vin', 'anoFabricacao', 'anoModelo', 'dataInstalacao', 'tipoMonitoramento',
    'marca', 'modelo', 'cor', 'descricao', 'frota', 'tipo', 'usuarioCriacao',  
    'proprietario'
]

### 1.4 - Empty values or Little data inputed

#### 1.4.1 - Vehicle DataFrame

In [156]:
# DataFrame from vehicles that don't have NaN for all column
df_vehicles.drop(drop_list_vehicles, axis=1).sample(3)         # see the columns and structure of dataframe

,id,dataCadastrado,deletado,status,vin,anoFabricacao,anoModelo,placa,marca,modelo,cor,descricao,tipo,assistencia,usuarioCriacao,proprietarioId,proprietario
809,180519,1587659091672,False,Ativado,9BVN4B5AOYE668947,2000.0,2000.0,AIS1982,0,,VERMELHO,INTEGRADOR - 04.07.2020 - INADIMPLENTE CONFORM...,Caminhão,False,11426,195696,CLAUDIO RODRIGUES LEMOS
6085,126064,1542127558961,False,Ativado,9BFYCEHV3ABB59990,2010.0,2010.0,ASW9523,0,,VERMELHO,DANILO JESUS - 05/04/2019 - STATUS ALTERADO PA...,Caminhão,False,11426,169335,LUIZ FELIPE RAMALHO
5895,113473,1529002550685,False,Ativado,9BSR4X2A0X3513083,2007.0,2007.0,AJA2852,0,,BRANCO,ANA - 08-05-19 INADIMPLENTE CONFORME EMAIL DA...,Caminhão,False,11426,163880,EDEMAR SAMPAIO


In [157]:
# Unique value, you will drop this columns
df_vehicles['deletado'].value_counts()
df_vehicles['fuso'].value_counts()
df_vehicles['tipoMonitoramento'].value_counts()
df_vehicles['assistencia'].value_counts()
# 'vin', 'usuarioCriacao', 'proprietarioId' -> it's not important columns

# Including news columns to will be filtered or drop
drop_list_vehicles.extend(['deletado', 'fuso', 'tipoMonitoramento', 'assistencia',
                           'vin', 'usuarioCriacao', 'proprietarioId'])

#### 1.4.2 - Positions DataFrame

In [158]:
# Dataframe from positions
df_positions.drop(drop_list_positions, axis=1).sample(3)       # see the columns and structure of dataframe

,id,placa,assistencia,proprietarioId,motoristas,dispositivos,grupos
2067,161028,ASD2240,False,188830,[],"[{'id': 265775, 'fabricanteId': 132, 'fabrican...",[]
999,179668,ETG3782,False,195279,[],"[{'id': 283078, 'fabricanteId': 132, 'fabrican...",[]
1936,179061,AXT1142,False,194930,[],"[{'id': 282242, 'fabricanteId': 132, 'fabrican...",[]


In [159]:
# Seeing the columns that has a only one value or a empty list
print(df_positions.drop(drop_list_positions, axis=1)['assistencia'].value_counts())
print(df_positions.drop(drop_list_positions, axis=1)['motoristas'].value_counts())
print(df_positions.drop(drop_list_positions, axis=1)['grupos'].value_counts())
# Columns that don't have importance or values
    # grupos
    # motoristas
    # assistencia - there only one information FALSE

# adding columns above that not is important or empty in a list
drop_list_positions.extend(['grupos', 'assistencia', 'motoristas'])

False    2830
Name: assistencia, dtype: int64
[]    2830
Name: motoristas, dtype: int64
[]    2830
Name: grupos, dtype: int64


### 1.5 - Data Types and Changes

In [160]:
# df_veh_sel = df_vehicles.drop(drop_list_vehicles, axis=1).copy()
df_pos_sel = df_positions.drop(drop_list_positions, axis=1).copy()

#### 1.5.1 - Vehicle DataFrame

In [161]:
df_veh_sel.info()

NameError: name 'df_veh_sel' is not defined

In [ ]:
# as you can see above, there are some datatypes wrong, let's see to understand it
df_veh_sel.sample(3)

In [ ]:
# dataCadastro  -> it's a milisseconds data from 1970 ultil datastamp, you need change to undertandble simple date
# AnoFabricacao -> It's float you will convert to int
# AnoModelo     -> It's float you will convert to int

# converting float to int and replace NaN to 0
df_veh_sel['anoFabricacao'] = df_veh_sel['anoFabricacao'].astype(int64).copy()
df_veh_sel['anoModelo'] = df_veh_sel['anoModelo'].astype(int64).copy()
df_veh_sel['dataCadastrado'] = df_veh_sel['dataCadastrado'].apply(
    lambda x: datetime.datetime.fromtimestamp(x/1000).date()).copy()

#### 1.5.2 - Positions DataFrame

In [162]:
df_pos_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              2830 non-null   int64 
 1   placa           2830 non-null   object
 2   proprietarioId  2830 non-null   int64 
 3   dispositivos    2830 non-null   object
dtypes: int64(2), object(2)
memory usage: 88.6+ KB


Neste caso manteremos id e dispositivos

In [ ]:
df_pos_sel.sample(3)

There are in a [dispositivos] columns like a list and inside the list have a dict, we will keep [numeroStr] and [posicoes]

Get the numeroStr in the first inside the list and put on dataframe

In [ ]:
# numeroStr
df_pos_sel['numeroStr'] = df_pos_sel['dispositivos'].apply(lambda x: x[0]['numeroStr'])

# posicoes
df_pos_sel['posicoes'] = pd.DataFrame(df_pos_sel['dispositivos'].apply(lambda x: x[0]['posicoes']))

In [ ]:
df_pos_sel.head(3)

Coping the list in column [posicoes] and transforme in dataFrame to get the max() value than get the others values from the list that have the max.

Inside the posicoes you get the ['sequencia', 'dataEquipamento', 'latitude', 'longitude', 'endereco', 'numerostr'], because it's important columns, if the posicoes has empty insert 0 all the columns and concat, after that concat to original df

In [ ]:
df_pos = pd.DataFrame()
for i in df_pos_sel['posicoes']:
    try:
        df = pd.DataFrame(i)[['sequencia', 'dataEquipamento', 'latitude', 'longitude', 'endereco', 'numerostr']]
        df = df[df['sequencia'] == df['sequencia'].max()][0:1]
        df_pos = pd.concat([df_pos, df], axis=0, ignore_index=True)
    except KeyError:
        df = pd.DataFrame([['0', '0', '0', '0', '0', '0']], columns=['sequencia', 
                'dataEquipamento', 'latitude', 'longitude', 'endereco', 'numerostr'])
        df_pos = pd.concat([df_pos, df], axis=0, ignore_index=True)
        
# Concat the filtered dataframe and original dataframe and dropped the columns that not is more important
df_pos_sel = pd.concat([df_pos_sel, df_pos], axis=1)
df_pos_sel = df_pos_sel.drop(['dispositivos', 'posicoes', 'numerostr',
            'proprietarioId', 'sequencia', 'numeroStr'], axis=1).copy()

In [ ]:
# It's ok the shape is equal
df_pos.shape, df_pos_sel.shape

In [ ]:
# dataEquipamento -> let's change to date and after to string
df_pos_sel['dataEquipamento'] = df_pos_sel['dataEquipamento'].astype(int64).copy()
df_pos_sel['dataEquipamento'] = df_pos_sel['dataEquipamento'].apply(
    lambda x: datetime.datetime.fromtimestamp(x/1000).date()).copy()

# Converting object to float, because the map geomap need to find it.
df_pos_sel['latitude'] = df_pos_sel['latitude'].astype(float).copy()
df_pos_sel['longitude'] = df_pos_sel['longitude'].astype(float).copy()

# Get the date and hour in exactly time request
df_pos_sel['date_request'] = datetime.date.today()
df_pos_sel['time_request'] = datetime.datetime.now().hour

In [ ]:
df_pos_sel.head()

In [ ]:
df_pos_sel.info()

## 2.0 - Insert Database

### 2.1 - Connection DataBase

This action mean that you create a connection with a existent database or create a new if not exist
It's keep the connection like a cursor than with this variable you use to send command like a insert, delete, update, etc.

In [756]:
# Create connection to create_database and table
conn = sqlite3.connect("tracker.db")
cursor = conn.cursor()

# Create engine "connection" from API sqlalchemy to insertion data
engine = create_engine('sqlite:////home/python/PycharmProjects/Projects/tracker/tracker.db', echo=False)

### 2.2 - Create Table

Criar tabelas manualmente 
pensar  em fazer o relacionamento entre id das duas tabelas

In [757]:
query_vehicles = '''CREATE TABLE IF NOT EXISTS vehicles (
            id INTEGER, dataCadastrado TEXT, status TEXT,
            anoFabricacao INTEGER, anoModelo INTEGER, placa TEXT,
            marca TEXT, modelo TEXT, cor TEXT, descricao TEXT, tipo TEXT,
            proprietario TEXT)
            '''
create_table(cursor, query_vehicles)


query_positions = '''CREATE TABLE IF NOT EXISTS positions (
            id INTEGER, placa TEXT, dataEquipamento, latitude REAL,
            longitude REAL, endereco TEXT)
            '''

create_table(cursor, query_positions)

### 2.3 - Insertion DataBase

In [759]:
# Run the insertion if is not exists table, else, show the error, remember, 
# the sequence of columns must to be equal by table, else, the information 
# will be change and replaced the wrong place.
df_veh_sel.to_sql('vehicles', con=engine, if_exists='append', index=False)
df_pos_sel.to_sql('positions', con=engine, if_exists='append', index=False)

In [ ]:
# Each positions insertion have 340Kb == 0.340MB 
# If you have insertion each 30 minutos, you will have 16.4MB/day == 490MB/month

### 2.2 - DataBase Select Query

In [35]:
select_cur = pd.DataFrame(cursor.execute(
            'select * from positions').fetchall(), columns=df_pos_sel.columns)

## 3.0 - API Request

In [29]:
# API to request data from users

In [779]:
# This picke just to continue the next steps, without loading all the codes

#with open('df_position_02_03', 'wb') as file:
#    pickle.dump(df_pos_sel, file)    

In [6]:
# Streamlit

#with open('df_position_02_03', 'rb') as file:
#    df_position = pickle.load(file)
#
#st.title("Veículos SegTruck")
#
#st.write(f"""
### Posições dos Veículos
#""")
#
#date = st.date_input('Selecione a data')
#
#
#data = df_position[df_position['dataEquipamento'] == date]
#f"""{data.shape[0]} Veículos no mapa."""
#st.map(data)

